# 神经网络：对数几率回归

实验内容：

使用肿瘤分类数据集
1. 完成对数几率回归
2. 使用梯度下降求解模型参数
3. 绘制模型损失值的变化曲线
4. 调整学习率和迭代轮数，观察损失值曲线的变化
5. 按照给定的学习率和迭代轮数，初始化新的参数，绘制新模型在训练集和测试集上损失值的变化曲线，完成表格内精度的填写

## 1. 导入数据集

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

In [ ]:
data = pd.read_csv('data/Breast_Cancer_Wisconsin/data')
print(data.shape)

In [ ]:
data = data.values 
data_x = data[:,2:-1]
data_y = data[:,1:2]
data_y = np.reshape(data_y,(-1))

data_y[data_y == 'M'] = 0
data_y[data_y == 'B'] = 1

print(data_x.shape)
print(data_y.shape)

data_x = data_x.astype(np.float64)
data_y = data_y.astype(np.float64)

选择40%的数据作为测试集，60%作为训练集

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data_x, data_y, test_size = 0.4, random_state = 32)

In [ ]:
trainX.shape, trainY.shape, testX.shape, testY.shape

## 2. 数据预处理

In [ ]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
trainX = s.fit_transform(trainX)
testX = s.transform(testX)

## 3. 定义神经网络

### 3.1 参数初始化

In [ ]:
def initialize(m):
    '''
    初始化参数W和参数b
    '''
    np.random.seed(32)
    W = np.random.normal(size = (m, )) * 0.01
    b = np.zeros((1, ))
    return W, b

### 3.2 前向传播

In [ ]:
def linear_combination(X, W, b):
    '''
    完成Z = XW + b的计算
    '''
    # YOUR CODE HERE
    Z = np.dot(X, W) + b
    return Z

接下来实现激活函数$\rm sigmoid$

In [ ]:
def my_sigmoid(x):
    '''
    simgoid 1 / (1 + exp(-x))
    '''
    # YOUR CODE HERE
    activations = 1 / (1 + np.exp(-x))
    return activations

In [ ]:
from scipy.special import expit

In [ ]:
def sigmoid(X):
    return expit(X)

In [ ]:
def forward(X, W, b):
    '''
    完成输入矩阵X到最后激活后的预测值y_pred的计算过程
    '''
    # 求Z
    # YOUR CODE HERE
    Z = linear_combination(X, W, b)
    # 求激活后的预测值
    # YOUR CODE HERE
    y_pred = my_sigmoid(Z)
    return y_pred

In [ ]:
def logloss(y_true, y_pred):
    '''
    给定真值y，预测值y_hat，计算对数损失并返回
    '''
    y_hat = np.clip(y_pred, 1e-10, 1 - 1e-10)
    # 求解对数损失
    # YOUR CODE HERE
    loss = -np.mean(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat))
    return loss

### 3.3 反向传播

我们接下来要完成损失函数对参数的偏导数的计算

In [ ]:
def compute_gradient(y_true, y_pred, X):
    '''
    给定预测值y_pred，真值y_true，传入的输入数据X，计算损失函数对参数W的偏导数的导数值dW，以及对b的偏导数的导数值db
    '''
    # 求损失函数对参数W的偏导数的导数值
    # YOUR CODE HERE
    dW = np.dot(X.T, y_pred - y_true) / len(y_true)
    # 求损失函数对参数b的偏导数的导数值
    # YOUR CODE HERE
    db = np.mean(y_pred - y_true)
    return dW, db

### 3.4 参数更新
给定学习率，结合上一步求出的偏导数，完成梯度下降的更新公式

In [ ]:
def update(W, b, dW, db, learning_rate):
    '''
    梯度下降，给定参数W，参数b，以及损失函数对他们的偏导数，使用梯度下降更新参数W和参数b
    '''
    W -= learning_rate * dW
    # 对参数b进行更新
    # YOUR CODE HERE
    b -= learning_rate * db

我们来完成整个反向传播和更新参数的函数

In [ ]:
def backward(y_true, y_pred, X, W, b, learning_rate):
    '''
    反向传播，包含了计算损失函数对各个参数的偏导数的过程，以及梯度下降更新参数的过程
    '''
    dW, db = compute_gradient(y_true, y_pred, X)
    update(W, b, dW, db, learning_rate)

## 4. 训练函数的编写

我们已经实现了完成训练需要的子函数，接下来就是组装了

In [ ]:
def train(trainX, trainY, testX, testY, W, b, epochs, learning_rate = 0.01, verbose = False):
    '''
    训练，我们要迭代epochs次，每次迭代的过程中，做一次前向传播和一次反向传播
    同时记录训练集和测试集上的损失值，后面画图
    '''
    training_loss_list = []
    testing_loss_list = []
    for i in range(epochs):
        # 计算训练集前向传播得到的预测值
        # YOUR CODE HERE
        train_pred = forward(trainX, W, b)
        # 计算当前训练集的损失值
        # YOUR CODE HERE
        training_loss = logloss(trainY, train_pred)
        # 计算测试集前向传播得到的预测值
        # YOUR CODE HERE
        test_pred = forward(testX, W, b)
        # 计算当前测试集的损失值
        # YOUR CODE HERE
        testing_loss = logloss(testY, test_pred)
        if verbose == True:
            print('epoch %s, training loss:%s'%(i + 1, training_loss))
            print('epoch %s, testing loss:%s'%(i + 1, testing_loss))
            print()
        training_loss_list.append(training_loss)
        testing_loss_list.append(testing_loss)
        
        # 反向传播更新参数
        # YOUR CODE HERE
        dW, db = compute_gradient(trainY, train_pred, trainX)
        update(W, b, dW, db, learning_rate)

    return training_loss_list, testing_loss_list

## 5. 绘制模型损失值变化曲线

In [ ]:
def plot_loss_curve(training_loss_list, testing_loss_list):
    '''
    绘制损失值变化曲线
    '''
    plt.figure(figsize = (10, 6))
    plt.plot(training_loss_list, label = 'training loss')
    plt.plot(testing_loss_list, label = 'testing loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()

## 6. 预测

In [ ]:
def predict(X, W, b):
    '''
    预测，调用forward函数完成神经网络对输入X的计算，然后完成类别的划分，大于0.5的变为1，小于等于0.5的变为0
    '''
    # YOUR CODE HERE
    y_pred = forward(X, W, b)
    prediction = (y_pred > 0.5).astype(int)    
    return prediction

## 7. 训练一个神经网络

我们的学习率是0.01，迭代200轮

In [ ]:
W, b = initialize(trainX.shape[1])
training_loss_list, testing_loss_list = train(trainX, trainY, testX, testY, W, b, 200, 0.01)

计算测试集精度

In [ ]:
prediction = predict(testX, W, b)
accuracy_score(testY, prediction)  # 0.9649122807017544

绘制损失值变化曲线

In [ ]:
plot_loss_curve(training_loss_list, testing_loss_list)

# test：初始化新的参数，学习率和迭代轮数按下表设置，绘制其训练集和测试集损失值的变化曲线，完成表格内精度的填写

###### 双击此处填写

学习率|迭代轮数|测试集精度
-|-|-
0.0001|200|0.8859649122807017
0.1|1000|0.9736842105263158

In [ ]:
# YOUR CODE HERE
W, b = initialize(trainX.shape[1])
training_loss_list, testing_loss_list = train(trainX, trainY, testX, testY, W, b, 200, 0.0001)
prediction = predict(testX, W, b)
print(accuracy_score(testY, prediction))
plot_loss_curve(training_loss_list, testing_loss_list)

In [ ]:
# YOUR CODE HERE
W, b = initialize(trainX.shape[1])
training_loss_list, testing_loss_list = train(trainX, trainY, testX, testY, W, b, 200, 0.1)
prediction = predict(testX, W, b)
print(accuracy_score(testY, prediction))
plot_loss_curve(training_loss_list, testing_loss_list)